In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, mean_absolute_error

df = pd.read_excel('City-Specific Data/Chicago.xlsx')
df = df.dropna()

temperature = df['Temperature'].values.reshape(-1, 1)
energy_demand = df['EnergyDemand'].values.reshape(-1, 1)

scaler_temp = MinMaxScaler()
scaler_energy = MinMaxScaler()
temperature_normalized = scaler_temp.fit_transform(temperature)
energy_demand_normalized = scaler_energy.fit_transform(energy_demand)

def create_sequences(data, seq_length):
    sequences = []
    targets = []
    for i in range(len(data) - seq_length):
        seq = data[i:i+seq_length]
        target = data[i+seq_length]
        sequences.append(seq)
        targets.append(target)
    return np.array(sequences), np.array(targets)

seq_length = 24
X, y = create_sequences(temperature_normalized, seq_length)
y = energy_demand_normalized[seq_length:]

kf = KFold(n_splits=5, shuffle=True, random_state=42)

mse_scores = []
mae_scores = []
mape_scores = []

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    non_zero_indices = y_true != 0
    return np.mean(np.abs((y_true[non_zero_indices] - y_pred[non_zero_indices]) / y_true[non_zero_indices])) * 100

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model = Sequential([
        LSTM(50, activation='relu', input_shape=(seq_length, 1)),
        Dense(1)
    ])

    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

    history = model.fit(X_train, y_train, epochs=25, batch_size=32, validation_split=0.1, verbose=1)

    loss = model.evaluate(X_test, y_test)
    print(f"Fold Test loss: {loss}")

    predictions_normalized = model.predict(X_test)

    predictions = scaler_energy.inverse_transform(predictions_normalized)
    y_test_original = scaler_energy.inverse_transform(y_test)

    mse = mean_squared_error(y_test_original, predictions)
    mae = mean_absolute_error(y_test_original, predictions)
    mape = mean_absolute_percentage_error(y_test_original, predictions)
    mse_scores.append(mse)
    mae_scores.append(mae)
    mape_scores.append(mape)
    print(f"Fold Mean Squared Error: {mse}")
    print(f"Fold Mean Absolute Error: {mae}")
    print(f"Fold Mean Absolute Percentage Error: {mape}")

average_mse = np.mean(mse_scores)
average_mae = np.mean(mae_scores)
average_mape = np.mean(mape_scores)
print(f"Average Mean Squared Error: {average_mse}")
print(f"Average Mean Absolute Error: {average_mae}")
print(f"Average Mean Absolute Percentage Error: {average_mape}")

plt.figure(figsize=(12, 6))
plt.plot(scaler_energy.inverse_transform(y_test[:100]), label='Actual')
plt.plot(predictions[:100], label='Predicted')
plt.title('Predicted vs Actual Energy Demand for Chicago (Last Fold)')
plt.xlabel('Time')
plt.ylabel('Energy Demand')
plt.legend()
plt.show()


Epoch 1/25


C:\Users\rushi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1133/1133 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - loss: 2.0657e-04 - val_loss: 8.8402e-05
Epoch 2/25
1133/1133 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - loss: 1.6103e-04 - val_loss: 8.2902e-05
Epoch 3/25
1133/1133 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - loss: 1.2522e-04 - val_loss: 6.5082e-05
Epoch 4/25
1133/1133 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - loss: 1.4259e-04 - val_loss: 5.3192e-05
Epoch 5/25
1133/1133 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - loss: 1.3930e-04 - val_loss: 5.0439e-05
Epoch 6/25
1133/1133 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - loss: 9.6636e-05 - val_loss: 5.1142e-05
Epoch 7/25
1133/1133 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - loss: 1.1508e-04 - val_loss: 5.0340e-05
Epoch 8/25
1133/1133 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - loss: 8.4447e-05 - val_loss: 4.6586e-05
Epoch 9/25
1133/1133 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - loss: 9.6612e-05 - val_loss: 5.6560e-05
Epoch 10/25
1133/1133 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - loss: 7.8591e-05 - val_loss: 4.7403e-05
Epoch 11/25
1133/1133 ━━━━━━━━━━━━━━━━━━━━ 8s 7m

C:\Users\rushi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1133/1133 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - loss: 3.0570e-04 - val_loss: 6.5229e-05
Epoch 2/25
1133/1133 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - loss: 9.5165e-05 - val_loss: 5.0139e-05
Epoch 3/25
1133/1133 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - loss: 9.6693e-05 - val_loss: 5.0654e-05
Epoch 4/25
1133/1133 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - loss: 8.5225e-05 - val_loss: 4.7058e-05
Epoch 5/25
1133/1133 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - loss: 8.2675e-05 - val_loss: 5.4946e-05
Epoch 6/25
1133/1133 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - loss: 7.7225e-05 - val_loss: 5.5131e-05
Epoch 7/25
1133/1133 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - loss: 7.9037e-05 - val_loss: 4.4749e-05
Epoch 8/25
1133/1133 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - loss: 7.8649e-05 - val_loss: 5.1601e-05
Epoch 9/25
1133/1133 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - loss: 7.4919e-05 - val_loss: 5.3313e-05
Epoch 10/25
1133/1133 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - loss: 7.4349e-05 - val_loss: 4.4674e-05
Epoch 11/25
1133/1133 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms

...

C:\Users\rushi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1133/1133 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - loss: 2.1514e-04 - val_loss: 1.3687e-04
Epoch 2/25
1133/1133 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - loss: 1.2628e-04 - val_loss: 7.1801e-05
Epoch 3/25
1133/1133 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - loss: 1.3510e-04 - val_loss: 6.4796e-05
Epoch 4/25
1133/1133 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - loss: 1.3901e-04 - val_loss: 6.1910e-05
Epoch 5/25
1133/1133 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - loss: 8.2888e-05 - val_loss: 5.1706e-05
Epoch 6/25
1133/1133 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - loss: 9.8217e-05 - val_loss: 5.4557e-05
Epoch 7/25
1133/1133 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - loss: 9.0351e-05 - val_loss: 5.1882e-05
Epoch 8/25
1133/1133 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - loss: 1.0446e-04 - val_loss: 4.8730e-05
Epoch 9/25
1133/1133 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - loss: 9.4801e-05 - val_loss: 4.9096e-05
Epoch 10/25
1133/1133 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - loss: 9.7089e-05 - val_loss: 4.8935e-05
Epoch 11/25
1133/1133 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/

C:\Users\rushi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1133/1133 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - loss: 2.2829e-04 - val_loss: 5.0848e-05
Epoch 2/25
1133/1133 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - loss: 1.0452e-04 - val_loss: 5.4321e-05
Epoch 3/25
1133/1133 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - loss: 1.1665e-04 - val_loss: 4.9311e-05
Epoch 4/25
1133/1133 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - loss: 9.4242e-05 - val_loss: 7.9033e-05
Epoch 5/25
1133/1133 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - loss: 1.6745e-04 - val_loss: 9.1128e-05
Epoch 6/25
1133/1133 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - loss: 1.0639e-04 - val_loss: 5.0423e-05
Epoch 7/25
1133/1133 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - loss: 8.3486e-05 - val_loss: 5.8030e-05
Epoch 8/25
1133/1133 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - loss: 1.0282e-04 - val_loss: 4.9922e-05
Epoch 9/25
1133/1133 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - loss: 8.9187e-05 - val_loss: 5.5986e-05
Epoch 10/25
1133/1133 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - loss: 7.9572e-05 - val_loss: 4.6948e-05
Epoch 11/25
1133/1133 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/

NameError: name 'plt' is not defined